In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2


import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import utils


In [2]:
dask.config.set(**{'array.slicing.split_large_chunks': True})

## Data

Data for this is from https://gmd.copernicus.org/articles/11/1133/2018/ CDRMIP data, where pi-CO2pulse is the 100GtC pulse and piControl is the control

### Notes on data
ACCESS: 10 years of pi control on the co2pulse run; take G 10 years in

GFDL: pi control run for 100 years before pulse, co2pulse run starts at 100 years in without any picontrol data

UKESM1: 10 years of pi control on the co2pulse run; take G 10 years in

MIROC: co2pulse starts 10 years after pi control, but doesn't include that in its data

NORESM2: picontrol starts in 1851 and co2pulse starts in 1850, both should start in 1851 according to documentation

CanESM5 (p2): picontrol is at 5601, but was branched at 5301; co2pulse was branched at 5401;line up by moving picontrol dt to start at 5301

In [4]:
model_run_pulse_dict = utils.model_run_pulse_dict
model_run_1pct_dict = utils.model_run_1pct_dict
model_run_control_dict = utils.model_run_esm_picontrol_dict


In [5]:
#define our output grid size

ds_out = xr.Dataset(
    {
        "lat": (["lat"], np.arange(-89.5, 90.5, 1.0)),
        "lon": (["lon"], np.arange(0, 360, 1)),
        "lat_b": (["lat_b"], np.arange(-90.,91.,1.0)),
        "lon_b":(["lon_b"], np.arange(.5, 361.5, 1.0))
    }
)

In [7]:
A = utils.find_area(ds_out)

In [8]:
ds_control = {}
ds_pulse = {}
G = {}

# anom_control = {}
# anom_pulse = {}
# anom_G = {}
for m1 in model_run_pulse_dict.keys():
    if m1 == 'CANESM5_r1p2' or m1 == 'CANESM5_r2p2' or m1 == 'CANESM5_r3p2':
        m2 = 'CANESM5_r1p2'
    else:
        m2 = m1
    print(m2, m1)
    ds_control[m1], ds_pulse[m1], G[m1] = utils.import_regrid_calc(f'cmip6_data/tas_Amon_{model_run_control_dict[m2]}', 
                                                 f'cmip6_data/tas_Amon_{model_run_pulse_dict[m1]}', 
                                                 ds_out, variable = 'tas', m = m1, pulse_size = 100, replace_xy = True)
#     anom_control[m], anom_pulse[m], anom_G[m] = utils.import_regrid_calc(f'cmip6_data/tas_Amon_{model_run_control_dict[m]}', 
#                                                  f'cmip6_data/tas_Amon_{model_run_pulse_dict[m]}', 
#                                                  ds_out, variable = 'tas', pulse_size = 100, anomaly = True)

UKESM1_r1 UKESM1_r1


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


MIROC MIROC


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


NORESM2 NORESM2


...

/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r1p2 CANESM5_r2p2


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


CANESM5_r1p2 CANESM5_r3p2


/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(
/home/emfreese/anaconda3/envs/gchp/lib/python3.9/site-packages/xesmf/frontend.py:555: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  ds_out = xr.apply_ufunc(


In [9]:
# ds_test_out = xr.Dataset(
#     {
#         "lat": (["lat"], np.arange(-89.375, 90.625, 1.25)),
#         "lon": (["lon"], np.arange(0.9375, 360.9375, 1.875)),
#         "lat_b": (["lat_b"], np.arange(-90.,91.25,1.25)),
#         "lon_b":(["lon_b"], np.arange(0, 361.875, 1.875))
#     }
# )
# A_test = find_area(ds_test_out)
# m = 'CANESM5_r1p2'
# ds_control = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_control_dict[m]}', preprocess = combined_preprocessing, use_cftime=True)
# ds_pulse = xr.open_mfdataset(f'cmip6_data/tas_Amon_{model_run_pulse_dict[m]}', preprocess = combined_preprocessing, use_cftime=True)
# [k for k in zip(ds_control.attrs.values(), ds_pulse.attrs.values()) if k[0]!=k[1]]
# A_test = A_test.rename({'lat':'y','lon':'x'})
# fig,ax = plt.subplots(figsize = [15,5])
# plt.plot(np.arange(0,len(ds_pulse.time)), (ds_pulse - ds_control)['tas'].weighted(A_test).mean(dim = ['x','y']))
# plt.plot(np.arange(0,len(ds_cdr.time)), -(ds_cdr - ds_control)['tas'].weighted(A_test).mean(dim = ['x','y']))
# plt.plot(np.arange(0,len(ds_pulse.time)), 0.5*(ds_pulse - ds_cdr)['tas'].weighted(A_test).mean(dim = ['x','y']))


# #plt.plot(np.arange(0,len(ds_pulse.time)), ds_pulse['tas'].weighted(A_test).mean(dim = ['x','y']))
# plt.axvline(x = 120, color = 'k', linewidth = 2)
# plt.xlim(0,500)


In [10]:
base = datetime.datetime.strptime("2000", "%Y")
for m in G.keys():
    G[m]['year'] = [base + relativedelta(years=x) for x in range(len(G[m]['year']))]
   # anom_G[m]['year'] = [base + relativedelta(years=x) for x in range(len(anom_G[m]['year']))]

In [11]:
G_ds = xr.concat([G[m] for m in G.keys()], pd.Index([m for m in G.keys()], name='model'))
#anom_G_ds = xr.concat([anom_G[m] for m in anom_G.keys()], pd.Index([m for m in anom_G.keys()], name='model'))

In [12]:
G_mean_ds = G_ds.mean(dim = 'model')
#anom_G_mean_ds = anom_G_ds.mean(dim = 'model')

## Save Green's Functions

In [14]:
G_ds.to_netcdf('Outputs/G_ds.nc4')
G_mean_ds.to_netcdf('Outputs/G_mean_ds.nc4')

# anom_G_ds.to_netcdf('Outputs/anom_G_ds.nc4')
# anom_G_mean_ds.to_netcdf('Outputs/anom_G_mean_ds.nc4')